# ECE449: Individaul Project
Jie Wang

Nov. 11, 2023

## Project Description

In this project, you need to solve an image classification task using coil-20-proc dataset.

This dataset consists of 1,440 grayscale images of 20 objects (72 images per object). 
- Half of each category is for training and half for testing. (e.g. 36 images for training and another 36 images for testing).

Through this project, you can learn how to customize dataset loading, design deep models, and train and test models. 

This is an individual project, so avoid copying others' code. Additionally, you need to print the accuracy results during the training process. 

> You should use a Python Jupyter Notebook to write down the process and accuracy results.


## The COIL-20 (Columbia Object Image Library) dataset
a collection of grayscale images of 20 different objects. 

- The "proc" in "COIL-20-proc" indicates that the images in this dataset have been preprocessed. photographed on a motorized turntable against a black background to provide a 360-degree view. The objects are varied, including toys, household items, and office supplies, to provide a range of shapes and complexities.


The preprocessing typically involves normalizing the images in terms of scale and orientation, and the background is often removed to isolate the object of interest. This makes the dataset particularly useful for computer vision tasks like object recognition and classification, where consistency in the input data can significantly improve the performance of machine learning models.


# 1. Complete the custom dataset, and dataloader.
Fill in the code to complete the custom dataset and dataloader, you can refer to https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [9]:
import os
import random
import glob

import argparse
import logging

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

from matplotlib import pyplot as plt
from tqdm import tqdm

class COIL20Dataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.images[index])
        label = self.labels[index]
        if self.transform:
            image = self.transform(image)
    def __len__(self):
        return len(self.images)


# Example transform
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# the COIL-20 images are stored in a directory as below:
# - coil-20-proc/
#       - 01/obj1__0.png, obj1__1.png, ...
#       - 02/...
#       - ...
#       - 20/....



In [24]:
def load_coil20_dataset(base_directory, transform, test_split=0.2):
    image_paths = []
    labels = []
    
    # Iterate over each category directory to collect image paths and labels
    for category in range(1, 21):  # Assuming 20 categories
        directory = f"{base_directory}/{str(category).zfill(2)}"
        category_image_paths = glob.glob(f"{directory}/*.png")
        # print(category_image_paths)
        category_labels = [category] * len(category_image_paths)  # Labels are the category number
        # print(category_labels)
        
        image_paths.extend(category_image_paths)
        labels.extend(category_labels)
    
    # Split the dataset into train and test
    split_idx = int(len(image_paths) * (1 - test_split))
    train_paths, test_paths = image_paths[:split_idx], image_paths[split_idx:]
    train_labels, test_labels = labels[:split_idx], labels[split_idx:]
    
    # Creating the dataset objects
    train_dataset = COIL20Dataset(train_paths, train_labels, transform)
    test_dataset = COIL20Dataset(test_paths, test_labels, transform)
    
    # Creating the dataloaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    return train_loader, test_loader, train_dataset, test_dataset

# Example transform
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Assuming the base directory is 'coil-20-proc' containing subdirectories '01', '02', ..., '20'
train_loader, test_loader, train_dataset, test_dataset = load_coil20_dataset('coil-20-proc', transform)


In [25]:
train_loader, test_loader, train_dataset, test_dataset

(<torch.utils.data.dataloader.DataLoader at 0x21b8bd746a0>,
 <__main__.COIL20Dataset at 0x21b8ae83250>)

In [30]:
print(train_dataset.__len__() ) 
print(test_dataset.__len__() ) 

1152
288


In [21]:
import matplotlib.pyplot as plt
import torchvision

def show_images_from_dataloader(dataloader):
    # Get a batch of training data
    images, labels = next(iter(dataloader))

    # Make a grid from batch
    out = torchvision.utils.make_grid(images)

    plt.figure(figsize=(15, 15))
    plt.imshow(out.numpy().transpose((1, 2, 0)))
    plt.title('Batch from dataloader')
    plt.axis('off')
    plt.show()

    # Print labels
    print('Labels:', labels.numpy())

# Example usage with your train_loader
show_images_from_dataloader(train_loader)


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [11]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


# 2. Implementing a Neural Network
Fill in the code to complete the custom model, you can refer to https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [12]:

class Classify_NN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(128*128,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,20)
        )
    def forward(self, x):
    # you should complete forward function
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# 3. Customize some arguments
Usually, this function is not used for jupyter notebook, as it violates the nature of interacive design.

Still, I set it mannually for the argument input. 

In [13]:


def get_config():
    parser = argparse.ArgumentParser()
    
    # you should complete arguments
    parser.add_argument()

    args = parser.parse_args()
    args.device = torch.device(args.device)

    return args

# 4. Train and Test the model.
Fill in the code to complete the training and testing, and print the results and losses. You can refer to https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html and https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [18]:



class Coil20_model:
    def __init__(self,trainset, testset, logger):
        self.train_loader = train_loader 
        self.test_loader = test_loader
        # Operate the method
        self.Mymodel = Classify_NN()
        self.criterion = nn.CrossEntropyLoss()  
        self.optimizer = torch.optim.Adam(self.Mymodel.parameters(), lr=0.001)  

    def _train(self, dataloader):
    # you should complete the training code, optimizer and loss function
        train_loss = 0
        n_correct = 0
        n_train = len(dataloader)

        for image,label in dataloader:
            pred_label = self.Mymodel(image)
            loss   = self.criterion(pred_label,label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            train_loss += loss.item * image.size(0)
            # n_correct += 

        return train_loss / n_train, n_correct / n_train

    def _test(self, dataloader):
        test_loss = 0
        n_correct = 0
        n_test    = len(dataloader)
        with torch.no_grad():
            for img, label in dataloader:
                pred_label = self.Mymodel(img)
                test_loss += self.criterion(pred_label,label).item() * img.size(0)
                if(pred_label == label):
                    n_correct += 1

        return test_loss / n_test, n_correct / n_test
    

    def _visualize(self, accuracy, val_accuracy):
        plt.plot(accuracy, label='accuracy')
        plt.plot(val_accuracy, label='val_accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.ylim([0, 1])
        plt.legend(loc='lower right')
        plt.show()

    def eval(self, dataloader):
        total_loss = 0.0
        total_correct = 0
        total_samples = 0

        with torch.no_grad():  # No need to track gradients for evaluation
            for batch_idx, (data, target) in enumerate(dataloader):
                data, target = data.to(device), target.to(device)
                outputs = self.Mymodel(data)
                loss = self.criterion(outputs, target)
                
                # Sum up batch loss
                total_loss += loss.item()

                # Get the index of the max log-probability (the predicted class label)
                _, predicted = torch.max(outputs.data, 1)
                total_samples += target.size(0)
                total_correct += (predicted == target).sum().item()

        # Calculate average loss and accuracy
        avg_loss = total_loss / len(dataloader)
        accuracy = total_correct / total_samples

        return avg_loss, accuracy

    def run(self):
    # you should complete run function for training and testing the model, and print the results and losses.
        train_loss, train_accuracy  = self._train(self.train_loader)
        print(f'Epoch {1}: Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}')

        print("finished training")
        test_loss, test_accuracy = self._test(self.test_loader)
        print(f'Epoch {1}: Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}')

        print("finished testing")



In [19]:
transform = transforms.Compose([
 transforms.Resize((128, 128)),
 transforms.ToTensor(),
 ])
train,test = load_coil20_dataset('coil-20-proc/',transform)
ece449_model = Coil20_model(train,test ,logger = logging.getLogger())  
ece449_model.run()  




TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [ ]:
# To save the model
torch.save(ece449_model.Mymodel.state_dict(), 'coil20_model.pth')

# To load the model
ece449_model.Mymodel.load_state_dict(torch.load('coil20_model.pth'))

In [ ]:
# Assuming test_data_loader is a DataLoader for new data
predictions = []
ece449_model.Mymodel.eval()  # Set model to evaluation mode
with torch.no_grad():
    for data in test_data_loader:
        data = data.to(args.device)
        output = model.Mymodel(data)
        pred = output.argmax(dim=1, keepdim=True)
        predictions.extend(pred.cpu().numpy())

# predictions now holds the predicted labels for the test data
